In [1]:
from sqlalchemy import create_engine
import datetime
import math
import pandas as pd
import numpy as np

In [2]:
def diagnose_counts(diaglist):
    #Load Data
    #engine = create_engine('postgresql:///goeg?host=/var/run/postgresql')
    #st = pd.read_sql_query("select * from stays where exit_date BETWEEN '2014-01-01'AND '2016-12-31'",con=engine).set_index('stay_id')
    #sex = pd.read_sql_query('select * from sex',con=engine)
    #age = pd.read_sql_query('select * from ag',con=engine)
    diagnoses = pd.read_sql_query('select * from diagnoses',con=engine)#,index_col='diagnose_id')
    
    diagnoses.sort_values('diagnose_id',axis=0,inplace=True)
    diagnoses.index = diagnoses['diagnose_id']
    print("The number of stays in this dataset is: " +str(st.shape))
    st['exit_date'] = st['exit_date'].apply(treat29feb)

    
    patient_info = st[['patient_no','sex_id','region_q_id']].groupby('patient_no').mean()
    print("The number of patinets in this dataset is "+ str(patient_info.shape))
    age['min_age'] = np.arange(0,95,5)
    age['max_age'] = np.concatenate((np.arange(4,94,5).reshape(18,1),np.array([[110]])))
    age.index = age.ag_id
    st['max_age'] = st['ag_id'].apply(lambda x: 5*x)
    st.loc[st.max_age==94,'max_age'] = 100
    
    df = st[st['pri_diag_id'].isin([i for i in diaglist])]
    
    datatime = datetime.datetime(2015, 1, 1)
    delta = datatime - pd.to_datetime(df['exit_date'])
    #Define an age group ID
    df['my_ag_id'] = np.ceil((df['max_age'] + delta.dt.days/365.25)/15).astype(int)
    df['my_ag_id'][df['my_ag_id']>6] = 6
    df = df.drop('ag_id', axis=1)
    
    #Count patients within a certain age, sex, region group
    df_counts =  df.groupby(['region_q_id', 'my_ag_id', 'sex_id']).size().reset_index()
    df_counts.rename({0: 'counts'}, axis='columns', inplace=True)
    print("the number of patients in this group is: ")
    print(df_counts['counts'].sum())
    df_counts = df_counts[~ df_counts['region_q_id'].isin([-1, 0])]
    index = pd.MultiIndex.from_frame(df_counts[['region_q_id', 'my_ag_id', 'sex_id']])
    test = df_counts.set_index(index)['counts']
    df_counts = test.unstack([1,2])
    print("And after removing patients outside austria: ")
    print(df_counts.sum().sum())
    regions = pd.read_sql_query('select * from regions',con=engine).set_index('region_id')
    df_counts = regions.join(df_counts,on='region_id')
    df_counts = df_counts.rename(columns={(1, 2):"F00diag",(2, 2):"F15diag",
                                    (3, 2):"F30diag",(4, 2):"F45diag",
                                    (5, 2):"F60diag",(6, 2):"F75diag",
                                    (1, 1):"M00diag",(2, 1):"M15diag",
                                    (3, 1):"M30diag",(4, 1):"M45diag",
                                    (5, 1):"M60diag",(6, 1):"M75diag",
                                    'descr':'Versorgungsregion'
                                   })
    return df_counts

In [4]:
DiagnoseBlocks = pd.read_pickle('./data/blocks.pkl')
DiagnoseBlocks[DiagnoseBlocks.description.str.contains('Diabetes')]

,ID,chapter,from_ICD,to_ICD,description,diagnosis_ids
32,32,4,E10,E14,Diabetes mellitus,"[350, 351, 352, 353, 354]"


In [ ]:
block = DiagnoseBlocks.iloc[32]
diaglist = block['diagnosis_ids']
disGroupName = block['description'].title().replace(' ', '')
print(disGroupName)
%time diag_c = diagnose_counts(diaglist)
filename = disGroupName + '.csv'
diag_c.to_csv(filename)